# P-tuning 实战

## Step1 导入相关包

In [2]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

## Step2 加载数据集

In [3]:
ds = Dataset.load_from_disk("./data/alpaca_data_zh/")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

## Step3 数据集预处理

In [4]:
tokenizer = AutoTokenizer.from_pretrained("D:/pretrained_model/models--Langboat--bloom-1b4-zh")
tokenizer

BloomTokenizerFast(name_or_path='D:/pretrained_model/models--Langboat--bloom-1b4-zh', vocab_size=46145, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [5]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [6]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [7]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

'Human: 解释为什么以下分数等同于1/4\n输入：4/16\n\nAssistant: 4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

In [8]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

## Step4 创建模型

In [9]:
model = AutoModelForCausalLM.from_pretrained("D:/pretrained_model/models--Langboat--bloom-1b4-zh", low_cpu_mem_usage=True)

## P-tuning

### PEFT Step1 配置文件

In [23]:
from peft import PromptEncoderConfig, TaskType, get_peft_model, PromptEncoderReparameterizationType

# # MLP
# config = PromptEncoderConfig(
#     task_type=TaskType.CAUSAL_LM,
#     num_virtual_tokens=10,
#     encoder_reparameterization_type=PromptEncoderReparameterizationType.MLP,
#     encoder_dropout=0.1
# )

# LSTM
config = PromptEncoderConfig(
    task_type=TaskType.CAUSAL_LM,
    num_virtual_tokens=10,
    encoder_reparameterization_type=PromptEncoderReparameterizationType.LSTM,
    encoder_dropout=0.1
)

### PEFT Step2 创建模型

In [24]:
model = get_peft_model(model, config)

In [25]:
config

PromptEncoderConfig(peft_type=<PeftType.P_TUNING: 'P_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, num_virtual_tokens=10, token_dim=2048, num_transformer_submodules=1, num_attention_heads=16, num_layers=24, encoder_reparameterization_type=<PromptEncoderReparameterizationType.LSTM: 'LSTM'>, encoder_hidden_size=2048, encoder_num_layers=2, encoder_dropout=0.1)

In [26]:
model.print_trainable_parameters()

trainable params: 193,030,144 || all params: 1,496,141,824 || trainable%: 12.9019


## Step5 配置训练参数

In [19]:
args = TrainingArguments(
    output_dir="./chatbot_ptuning",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1
)

## Step6 创建训练器

In [20]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

C:\Users\49207\AppData\Roaming\Python\Python311\site-packages\accelerate\accelerator.py:447: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


## Step7 模型训练

In [21]:
trainer.train()

  0%|          | 0/3357 [00:00<?, ?it/s]

{'loss': 2.9149, 'grad_norm': 4.4620137214660645, 'learning_rate': 4.985105749180816e-05, 'epoch': 0.0}
{'loss': 2.8197, 'grad_norm': 5.716224670410156, 'learning_rate': 4.9702114983616324e-05, 'epoch': 0.01}
{'loss': 2.6382, 'grad_norm': 1.7527867555618286, 'learning_rate': 4.9553172475424484e-05, 'epoch': 0.01}
{'loss': 2.5514, 'grad_norm': 3.1546542644500732, 'learning_rate': 4.940422996723265e-05, 'epoch': 0.01}
{'loss': 2.5955, 'grad_norm': 2.132143259048462, 'learning_rate': 4.925528745904081e-05, 'epoch': 0.01}
{'loss': 2.6094, 'grad_norm': 1.772666096687317, 'learning_rate': 4.910634495084897e-05, 'epoch': 0.02}
{'loss': 2.4354, 'grad_norm': 2.4859273433685303, 'learning_rate': 4.895740244265713e-05, 'epoch': 0.02}
{'loss': 2.4996, 'grad_norm': 2.068258285522461, 'learning_rate': 4.88084599344653e-05, 'epoch': 0.02}
{'loss': 2.3868, 'grad_norm': 2.2930712699890137, 'learning_rate': 4.865951742627346e-05, 'epoch': 0.03}
{'loss': 2.374, 'grad_norm': 1.2950072288513184, 'learning_

KeyboardInterrupt: 

## Step8 模型推理

In [22]:
model = model.cuda()
ipt = tokenizer("Human: {}\n{}".format("数学考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
print(tokenizer.decode(model.generate(**ipt, max_length=256, do_sample=True)[0], skip_special_tokens=True))

Human: 数学考试有哪些技巧？

Assistant: 在数学考试中，考生需要掌握的知识点非常之多，因此，在数学考试之前，建议考生先整理一份学习计划。计划可以让你对所学的所有科目内容形成一个概念性的认识。此外，计划还将帮助考生对考试中的每一个难点内容有一个更加清晰的认识。
